In [1]:
%reload_ext autoreload
%autoreload 2
from MultimodalDataset.GasData import GasDataProcessor
from MultimodalDataset.MedicalData import MedicalDataProcessor
from MultimodalDataset.ClimateData import ClimateDataProcessor
from src.utils import load_config, get_max_token_size

# For Climate Dataset

In [ ]:
import yaml

file_path = "config/default/climate.yaml"
# Load the existing YAML file
with open(file_path, 'r') as file:
    config = yaml.safe_load(file)

# Iterate over input_window and output_window from 1 to 7
for i in range(1, 8):
    # Update the values in the config
    config['dataset']['input_window'] = i
    config['dataset']['output_window'] = i
    config['dataset']['hf_repo'] = f"Howard881010/climate-{i}day"
    
    # Save the updated configuration back to the original file
    with open(file_path, 'w') as outfile:
        yaml.dump(config, outfile, default_flow_style=False)
    
    print(f"Saved configuration with input_window={i} and output_window={i}.")
    climate_cfg = load_config('config/default/climate.yaml')

    processor = ClimateDataProcessor(climate_cfg['dataset'])
    climate_dataset = processor.get_dataset()
    max_length = get_max_token_size(climate_dataset)
    processor.push_to_huggingface(climate_dataset)
    print(max_length)

print("All configurations saved to gas.yaml.")

In [ ]:
from datasets import load_dataset, concatenate_datasets, DatasetDict

data1 = load_dataset('Howard881010/climate-1day')
# data2 = load_dataset('Howard881010/climate-1day-mixed')
data3 = load_dataset('Howard881010/climate-2day')
# data4 = load_dataset('Howard881010/climate-2day-mixed')
data5 = load_dataset('Howard881010/climate-3day')
# data6 = load_dataset('Howard881010/climate-3day-mixed')
data7 = load_dataset('Howard881010/climate-4day')
# data8 = load_dataset('Howard881010/climate-4day-mixed')
data9 = load_dataset('Howard881010/climate-5day')
# data10 = load_dataset('Howard881010/climate-5day-mixed')
data11 = load_dataset('Howard881010/climate-6day')
# data12 = load_dataset('Howard881010/climate-6day-mixed')
data13 = load_dataset('Howard881010/climate-7day')
# data14 = load_dataset('Howard881010/climate-7day-mixed')

data_all = {}

for split in ['train', 'test', 'valid']:
    # data_all[split] = concatenate_datasets([data1[split], data2[split], data3[split], data4[split], data5[split], data6[split], data7[split], data8[split], data9[split], data10[split], data11[split], data12[split], data13[split], data14[split]])
    data_all[split] = concatenate_datasets([data1[split], data3[split], data5[split], data7[split], data9[split], data11[split], data13[split]])
    
data_dict = DatasetDict({
    'train': data_all['train'],
    'test': data_all['test'],
    'valid': data_all['valid']
})

data_dict.push_to_hub('Howard881010/climate')

# For Medical dataset

In [4]:
import yaml

file_path = "config/default/medical.yaml"
# Load the existing YAML file
with open(file_path, 'r') as file:
    config = yaml.safe_load(file)

# Iterate over input_window and output_window from 1 to 7
for i in range(1, 8):
    # Update the values in the config
    config['dataset']['input_window'] = i
    config['dataset']['output_window'] = i
    config['dataset']['hf_repo'] = f"Howard881010/medical-{i}day"
    
    # Save the updated configuration back to the original file
    with open(file_path, 'w') as outfile:
        yaml.dump(config, outfile, default_flow_style=False)
    
    print(f"Saved configuration with input_window={i} and output_window={i}.")
    medical_cfg = load_config('config/default/medical.yaml')

    processor = MedicalDataProcessor(medical_cfg['dataset'])
    medical_dataset = processor.get_dataset()
    max_length = get_max_token_size(medical_dataset)
    processor.push_to_huggingface(medical_dataset)
    print(max_length)

print("All configurations saved to medical.yaml.")

Saved configuration with input_window=1 and output_window=1.
Given the medical notes and heart rates of the first 1 day, predict the medical notes and heart rates of the next 1 day.Output the result **ONLY** in the following YAML format:
```
day_2_date:
day_2_medical_notes:
```


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/713 [00:00<?, ?B/s]

5469
All configurations saved to medical.yaml.
